In [1]:
from huggingface_hub import login


login(token= '')   #enter your creds

# Installations & Imports

In [2]:
%pip install \
    datasets \
    evaluate \
    rouge_score\
    loralib \
    evaluate \
    accelerate \
    bitsandbytes \
    trl \
    peft \
    -U --quiet

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import pandas as pd
import re
import numpy as np
import string
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
import evaluate

In [4]:
!pip install -q -U git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [6]:
import transformers

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", token=True)

# Quantization configuration
compute_dtype = torch.float16  # Use FP16 for memory efficiency
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,               # Enable 4-bit quantization
    bnb_4bit_quant_type="nf4",       # Quantization type (nf4 recommended)
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True   # Enable double quantization for efficiency
)

model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b-instruct",
    device_map="auto",
    torch_dtype=torch.float16,
    offload_folder="./offload"
)


# Optional model configurations
model.config.use_cache = False         # Disable caching (useful for training)
model.config.pretraining_tp = 1        # Pretraining tensor parallelism (adjust if needed)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# Set pad_token as end-of-sentence token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [9]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 6921720704
all model parameters: 6921720704
percentage of trainable model parameters: 100.00%


# Load 3 datasets

In [10]:
from datasets import load_dataset

dataset1 = load_dataset("Amod/mental_health_counseling_conversations")
dataset2 = load_dataset("mpingale/mental-health-chat-dataset")
dataset3 = load_dataset("heliosbrahma/mental_health_chatbot_dataset")

In [11]:
df1= dataset1["train"].data.to_pandas().dropna()
df2= dataset2["train"].data.to_pandas().dropna()
df3= dataset3["train"].data.to_pandas().dropna()

# Preprocess datasets

In [12]:
df1.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [13]:
df1.columns = ["question", "answer"]
df1

,question,answer
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...
...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit..."
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...


In [14]:
df2.head()

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,text
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971,<s>[INST] I have so many issues to address. I ...
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386,<s>[INST] I have so many issues to address. I ...
2,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Shakeeta TorresFaith Based Mental Health Couns...,https://counselchat.com/therapists/shakeeta-to...,Absolutely not. I strongly recommending worki...,2,3071,<s>[INST] I have so many issues to address. I ...
3,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Noorayne ChevalierMA, RP, CCC, CCAC, LLP (Mich...",https://counselchat.com/therapists/noorayne-ch...,Let me start by saying there are never too man...,2,2643,<s>[INST] I have so many issues to address. I ...
4,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Toni Teixeira, LCSWYour road to healing begins...",https://counselchat.com/therapists/toni-teixei...,I just want to acknowledge you for the courage...,1,256,<s>[INST] I have so many issues to address. I ...


In [15]:
# Just rename columns
df2 = df2[["questionText", "answerText"]]
df2.columns = ["question", "answer"]
df2

,question,answer
0,I have so many issues to address. I have a his...,It is very common for people to have multiple ...
1,I have so many issues to address. I have a his...,"I've never heard of someone having ""too many i..."
2,I have so many issues to address. I have a his...,Absolutely not. I strongly recommending worki...
3,I have so many issues to address. I have a his...,Let me start by saying there are never too man...
4,I have so many issues to address. I have a his...,I just want to acknowledge you for the courage...
...,...,...
2770,What are some difficulties that a counselor ca...,Although many clients have the capacity to be ...
2771,What are some difficulties that a counselor ca...,"I usually don't label a client as ""difficult"" ..."
2772,What are some difficulties that a counselor ca...,"Dang right! :)Heh heh, and correct me if I'm ..."
2773,What are some difficulties that a counselor ca...,"Yes, just like some relationships outside of o..."


In [16]:
df3.head()

,text
0,<HUMAN>: What is a panic attack?\n<ASSISTANT>:...
1,<HUMAN>: What are symptoms of panic attack vs....
2,<HUMAN>: What are the types of Mental Illness?...
3,<HUMAN>: What does mental-illness mean?\n<ASSI...
4,<HUMAN>: How can you treat mental illness?\n<A...


In [17]:
# Split single sample into Question & Answer

def split_qa(text):
    # below split returns ["", "question\n", "answer "]
    question, answer = re.split(r" ?<\w+>: ?", text)[1:]
    question = question.strip().replace("\n","")
    answer = answer.strip().replace("\n", "")
    return question, answer

df3["text"] = df3["text"].apply(split_qa)
df3[["question", "answer"]] = pd.DataFrame(data=[[q, a] for q, a in df3["text"].values])
df3.drop(columns=["text"], inplace=True)
df3


,question,answer
0,What is a panic attack?,Panic attacks come on suddenly and involve int...
1,What are symptoms of panic attack vs. anxiety ...,Panic attacks and anxiety attacks can share so...
2,What are the types of Mental Illness?,There are many different conditions that are r...
3,What does mental-illness mean?,Mental illness is a group of biological brain ...
4,How can you treat mental illness?,The treatment of mental illness is a multi-fac...
...,...,...
167,Who are you?,Hello there! I am your empathetic Mental Healt...
168,What is the helpline number for suicide preven...,I'm really sorry but I can't provide helpline ...
169,Which is the best mental health hospital in Ne...,"I'm really sorry, but I can't provide accurate..."
170,Who is the best hypnotherapist in London?,I understand that you're looking for a hypnoth...


In [18]:
# Merge datasets
df = pd.concat((df1, df2, df3)).sample(frac=1)
df

,question,answer
2251,She was raised by her abusive father and his w...,Was either parent abusive or violent toward yo...
1450,The last of my emotions belong to my pets. Tod...,It sounds to me like you have had a lot going ...
3269,My mother takes care of niece whom my sister a...,This sounds like a possible boundary issue. Bo...
3115,I have three children. The oldest is nine and ...,Congrats on being honest about the way you fee...
1498,I'm 15 and my girlfriend is 14. Am I a pedophi...,Wait what? Normal teenage experiences. A pe...
...,...,...
1390,I don’t love my sister. I would never wish her...,Hi. My guess is there's a lot of deep history ...
3417,I am in my early 20s and I still live with my ...,Hello. It sounds like you and your parents are...
1225,I am in a high stress position for a tech comp...,Hola. I understand. I worked 27 years in a sa...
293,I am pretty sure I have depression and anxiety...,"Talking with parents can be very challenging, ..."


In [19]:
def tokenize_function(row):
    # Tokenize the conversations
    row['input_ids'] = tokenizer(row["question"], padding="max_length", truncation=True, max_length = 128, return_tensors="pt").input_ids[0]

    # Assuming "answer" column is already a string, no need for conversion
    row['labels'] = tokenizer(row["answer"], padding="max_length", truncation=True, max_length = 128, return_tensors="pt").input_ids[0]

    return row


# Tokenize the DataFrame
tokenized_df = df.apply(tokenize_function, axis=1)
tokenized_df['input_ids'] = tokenized_df['input_ids'].apply(lambda x: x.tolist())
tokenized_df['labels'] = tokenized_df['labels'].apply(lambda x: x.tolist())

In [20]:
tokenized_df

,question,answer,input_ids,labels
2251,She was raised by her abusive father and his w...,Was either parent abusive or violent toward yo...,"[3066, 398, 5531, 431, 573, 29423, 3685, 273, ...","[14581, 2335, 2386, 29423, 379, 12648, 4419, 2..."
1450,The last of my emotions belong to my pets. Tod...,It sounds to me like you have had a lot going ...,"[487, 1143, 275, 491, 9278, 6445, 271, 491, 12...","[1140, 4914, 271, 454, 606, 299, 413, 618, 241..."
3269,My mother takes care of niece whom my sister a...,This sounds like a possible boundary issue. Bo...,"[2680, 3116, 2869, 1146, 275, 35858, 6088, 491...","[1182, 4914, 606, 241, 1777, 20405, 2645, 25, ..."
3115,I have three children. The oldest is nine and ...,Congrats on being honest about the way you fee...,"[52, 413, 1373, 1770, 25, 390, 13869, 304, 759...","[34371, 313, 1003, 5096, 544, 248, 818, 299, 1..."
1498,I'm 15 and my girlfriend is 14. Am I a pedophi...,Wait what? Normal teenage experiences. A pe...,"[52, 18, 88, 204, 1030, 273, 491, 13374, 304, ...","[17849, 637, 42, 4381, 23011, 22449, 5031, 25,..."
...,...,...,...,...
1390,I don’t love my sister. I would never wish her...,Hi. My guess is there's a lot of deep history ...,"[52, 794, 329, 95, 1163, 491, 6217, 25, 295, 6...","[5516, 25, 1814, 4191, 304, 629, 18, 94, 241, ..."
3417,I am in my early 20s and I still live with my ...,Hello. It sounds like you and your parents are...,"[52, 653, 272, 491, 2304, 204, 423, 94, 273, 2...","[9856, 25, 605, 4914, 606, 299, 273, 402, 3300..."
1225,I am in a high stress position for a tech comp...,Hola. I understand. I worked 27 years in a sa...,"[52, 653, 272, 241, 986, 4025, 2524, 312, 241,...","[36550, 25, 295, 1576, 25, 4381, 295, 3320, 20..."
293,I am pretty sure I have depression and anxiety...,"Talking with parents can be very challenging, ...","[52, 653, 2433, 1310, 295, 413, 9319, 273, 740...","[45361, 335, 3300, 418, 314, 829, 7607, 23, 83..."


In [21]:
# Remove texts and keep input_ids and labels
tokenized_df.drop(columns=["question", "answer"], inplace=True)

In [22]:
# import gc
# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.empty_cache()

In [23]:
# Split data into train & test
# train_df, test_df = train_test_split(tokenized_df, test_size=0.15)

# print(f"Training Size: {train_df.shape[0]} samples")
# print(f"Test Size: {test_df.shape[0]} samples")

In [24]:
from datasets import Dataset

# # Assuming `tokenized_df` is your pandas DataFrame
# train_dataset = Dataset.from_pandas(train_df)
# test_dataset = Dataset.from_pandas(test_df)

train_dataset = Dataset.from_pandas(tokenized_df)

# Configure PEFT Trainer

In [25]:
# Load LoRA configuration
peft_args = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
)

In [26]:
# Set training parameters
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
#     evaluation_strategy="epoch",
    optim="paged_adamw_32bit",
    save_steps=500,
    logging_steps=100,
    learning_rate=6e-5,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
#     max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [27]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_args,
    processing_class=tokenizer,
    args=training_params
)


# Training

In [28]:
trainer.train()

Step,Training Loss
100,2.558700
200,2.424400
300,2.285700
400,2.297500
500,2.204900
600,2.234500
700,2.137600
800,1.978300
900,2.022200
1000,1.919800


TrainOutput(global_step=6296, training_loss=1.5797556182783774, metrics={'train_runtime': 838.4553, 'train_samples_per_second': 15.018, 'train_steps_per_second': 7.509, 'total_flos': 6.410301619725926e+16, 'train_loss': 1.5797556182783774, 'epoch': 2.0})

# Save model & Publish

In [29]:
trainer.model.save_pretrained("./tiiuae-falcon-7b-instruct-Psychotherapist")
tokenizer.save_pretrained("./tiiuae-falcon-7b-instruct-Psychotherapist")

('./tiiuae-falcon-7b-instruct-Psychotherapist/tokenizer_config.json',
 './tiiuae-falcon-7b-instruct-Psychotherapist/special_tokens_map.json',
 './tiiuae-falcon-7b-instruct-Psychotherapist/tokenizer.json')

In [30]:
model.push_to_hub("Devmithun/tiiuae-falcon-7b-instruct-Psychotherapist")
tokenizer.push_to_hub("Devmithun/tiiuae-falcon-7b-instruct-Psychotherapist2")

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Devmithun/tiiuae-falcon-7b-instruct-Psychotherapist2/commit/42767019f5ba8f8f98ee03375946a5492567e4fc', commit_message='Upload tokenizer', commit_description='', oid='42767019f5ba8f8f98ee03375946a5492567e4fc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Devmithun/tiiuae-falcon-7b-instruct-Psychotherapist2', endpoint='https://huggingface.co', repo_type='model', repo_id='Devmithun/tiiuae-falcon-7b-instruct-Psychotherapist2'), pr_revision=None, pr_num=None)

# Inference

In [79]:
def generate_advice_for_counselor(patient_statement):
    """
    Generate actionable and empathetic advice for a counselor based on a patient's statement.

    Args:
        patient_statement (str): Symptoms or issues described by the counselor.

    Returns:
        str: Generated advice for the counselor.
    """
    # Define the conversation template
    messages = [
        {
            "role": "system",
            "content": (
                "You are a mental health expert who provides actionable advice to counselors. "
                "Based on the symptoms described by the patient, generate specific, empathetic, and actionable steps "
                "the counselor can take to help the patient. Avoid vague suggestions and focus on clear, practical solutions."
            ),
        },
        {"role": "user", "content": patient_statement},
    ]

    # Tokenize the input for the model
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True
    ).to(model.device)

    # Generate a response using the model
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=150,  # Limit token count for concise responses
        do_sample=False,  # Deterministic output
        temperature=0.3,  # Reduce randomness
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the response
    response_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
    advice = tokenizer.decode(response_tokens, skip_special_tokens=True)

    # Clean and remove redundant lines
    cleaned_advice = "\n".join(dict.fromkeys(advice.split("\n"))).strip()
    return cleaned_advice


# Test the function
patient_statement = """The patient reports feeling overwhelmed and has difficulty managing their daily tasks.
They often feel stuck and lack motivation to take action. What should I do to help them?"""

advice = generate_advice_for_counselor(patient_statement)

print(f'INPUT STATEMENT:\n{patient_statement}')
print(f'\n\nGenerated Advice:\n{advice}')


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


INPUT STATEMENT:
The patient reports feeling overwhelmed and has difficulty managing their daily tasks. 
They often feel stuck and lack motivation to take action. What should I do to help them?


Generated Advice:
Create a clear and specific action plan that integrates your advice into daily life. This may include breaking tasks into smaller steps, tracking progress, and celebrating successes. Additionally, provide regular check-ins to monitor progress and provide emotional support. Remember, each patient is unique, and what works for one person may not work for another. Be adaptable, empathetic, and creative in your approach.


In [80]:
question = """I'm always confused when making decisions, I cannot choose one option,
I must have only one option so I can make a decision, what are the causes of being
indecisive?"""

answer = generate_advice_for_counselor(question)

print(f'INPUT QUESTION:\n{question}')
print(f'\n\nModel Answer:\n{answer}')

INPUT QUESTION:
I'm always confused when making decisions, I cannot choose one option,
I must have only one option so I can make a decision, what are the causes of being
indecisive?


Model Answer:
When helping a patient with indecisiveness, consider the following: 1. Ask the patient about the specific reasons why they're having trouble choosing between options. 2. Encourage the patient to consider the potential outcomes of each option and weigh their importance. 3. Help the patient identify which options align with their values, needs, and long-term goals. 4. Provide concrete examples of how to make decisions, even if they're not the best fit. 5. Guide the patient to reflect on past experiences and identify patterns that may be affecting their indecisiveness. 6. Offer actionable advice on how to overcome indecisiveness, such as prioritizing the most important aspects, gathering more information, and weighing the pros and


In [81]:
question = """I feel like I don't exist and my body is not my own,
like if I'm sombody else observin me, what could be this disorder?"""

answer = generate_advice_for_counselor(question)

print(f'INPUT QUESTION:\n{question}')
print(f'\n\nModel Answer:\n{answer}')

INPUT QUESTION:
I feel like I don't exist and my body is not my own,
like if I'm sombody else observin me, what could be this disorder?


Model Answer:
It is essential to listen to the patient's statements and provide validation, reassurance, and empathy. It is also crucial to offer actionable advice that is clear, specific, and easy to understand. Avoid vague statements such as "you have your own thoughts" or "just feel better." Instead, provide concrete steps like "identify triggers" or "practice relaxation techniques." Encourage the patient to keep a journal to track progress and progressions. Additionally, provide resources, such as articles or books, to enhance self-understanding and self-care.
